In [102]:
from pathlib import Path
import pickle

import pandas as pd
import numpy as np
import lightgbm as lgb

In [40]:
models_dir_path = '../../../../models/'
model_metadata_path = '../../../../model_matadata/model_metadata.pkl'

In [41]:
def load_model(model_path):
    with open(model_path, "rb") as f:
        model = pickle.load(f)
    return model

In [42]:
models = [load_model(path) for path in Path(models_dir_path).glob("*")]

In [153]:
model_metadata = {'train_metrics': {key: 0} for key in model.best_score['train_sets'].keys()}
model_metadata = {'valid_metrics': {key: 0} for key in model.best_score['valid_sets'].keys()}
model_metadata['feature_name'] = models[0].feature_name()
model_metadata['feature_importance'] = np.array([])

for model in models:
    for key, value in model.best_score['train_sets'].items():
        model_metadata['train_metrics'][key] += value
        
    for key, value in model.best_score['valid_sets'].items():
        model_metadata['valid_metrics'][key] += value

    model_metadata['feature_importance'] = np.append(model_metadata['feature_importance'], model.feature_importance())

for key in model_metadata['valid_metrics']:
    model_metadata['valid_metrics'][key] = model_metadata['valid_metrics'][key]/len(models)
model_metadata['feature_importance'].reshape(-1, len(models[0].feature_name()))
model_metadata['feature_importance'] = np.mean(model_metadata['feature_importance'], axis=0)

In [154]:
model_metadata

{'valid_metrics': {'rmse': 57.160757333894935},
 'feature_name': ['age',
  'sex',
  'bmi',
  'bp',
  's1',
  's2',
  's3',
  's4',
  's5',
  's6'],
 'feature_importance': 21.6}

In [20]:
# Dump the objects
Path(model_metadata_path).parent.mkdir(
    parents=True, exist_ok=True
) 
with open(model_metadata_path, 'wb') as f:
  pickle.dump(model_metadata, f)